# Contents-based filtering algorithm for movielens 100k data
- reference: https://acodeforthought.wordpress.com/2016/12/29/building-a-recommender-system-on-user-user-collaborative-filtering-movielens-dataset/

In [1]:
import pandas as pd
import numpy as np
import math

## Data import
- movielens 100k dataset 으로부터 3가지 데이터 셋을 불러온다
    - **u.user**: 각 user에 대한 정보(demographic information about each user)
    - **u.item**: 각 영화에 대한 정보(description regarding each item, i.e., each movie)
    - **u.data**: 각 user의 영화에 대한 평가(rating of each user regarding an item)

In [2]:
# 각 데이터(user, item, rating)의 열 이름(column name)을 정의한다
user_cols = ['user id','age','gender','occupation','zip code']
item_cols = ['movie id','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance ','Sci-Fi','Thriller','War' ,'Western']
rating_cols = ['user id','movie id','rating','timestamp']

In [3]:
# pandas의 read_csv() 함수를 활용해 각 데이터셋을 불러온다
users = pd.read_csv('ml-100k/u.user', sep = '|', names = user_cols, encoding = 'latin-1')       
items = pd.read_csv('ml-100k/u.item', sep = '|', names = item_cols, encoding = 'latin-1')
ratings = pd.read_csv('ml-100k/u.data', sep = '\t', names = rating_cols, encoding = 'latin-1')

### Dataset description

In [4]:
print(users.shape)     # 943 users in total
print(items.shape)     # 1682 items (i.e., movies) in total
print(ratings.shape)   # 100000 ratings in total

(943, 5)
(1682, 24)
(100000, 4)


In [5]:
print(users.head())    # users dataframe의 첫 5 행의 데이터

   user id  age gender  occupation zip code
0        1   24      M  technician    85711
1        2   53      F       other    94043
2        3   23      M      writer    32067
3        4   24      M  technician    43537
4        5   33      F       other    15213


In [6]:
print(items.head())    # items dataframe의 첫 5 행의 데이타

   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Childrens   ...     Fantasy  Film-Noir  Horror  \
0          0          1          1   ...           0          0       0   
1       

In [7]:
print(ratings.head())    # ratings dataframe의 첫 5 행의 데이타

   user id  movie id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


## Data preprocessing

### train-test split
- 학습 데이터와 검증 데이터로 전체 데이터셋을 나눈다
- user id를 기준으로 ratings 데이터를 sorting한 후 99832번째 관측치까지는 학습 데이터로, 99833번째 관측치부터는 검증 데이터로 할당한다
- 그 결과 user id 1부터 942까지의 관측치는 학습 데이터가 되고, user id 943과 관련된 관측치는 검증 데이터가 된다
- 즉, 우리는 영화 간의 유사도와 943번째 user의 선호를 통해 그가 좋아할 만한 영화를 예측해 본다

In [17]:
ratings_train = (ratings.sort_values('user id'))[:99832]    
ratings_test = (ratings.sort_values('user id'))[99833:]

In [18]:
print(ratings_train.shape)
print(ratings_test.shape)

(99832, 4)
(167, 4)


In [10]:
print(ratings_train.head())
print()
print(ratings_train.tail())
print()
print(ratings_test.head())
print()
print(ratings_test.tail())

       user id  movie id  rating  timestamp
66567        1        55       5  875072688
62820        1       203       4  878542231
10207        1       183       5  875072262
9971         1       150       5  876892196
22496        1        68       4  875072688

       user id  movie id  rating  timestamp
73676      942       479       4  891283118
67222      942       604       4  891283139
95675      942       478       5  891283017
85822      942       659       5  891283161
68192      942       487       4  891282985

       user id  movie id  rating  timestamp
91841      943       132       3  888639093
91810      943       204       3  888639117
77956      943        94       4  888639929
87415      943        53       3  888640067
77609      943       124       3  875501995

       user id  movie id  rating  timestamp
96823      943       427       4  888639147
70902      943        12       5  888639093
84518      943       284       2  875502192
72321      943        62     

In [23]:
ratings_test = ratings_test.sort_values(by = ['rating'], ascending = False)    # rating을 기준으로 내림차순 정렬을 한다
ratings_test

,user id,movie id,rating,timestamp
80302,943,721,5,888639660
88402,943,614,5,888639351
95175,943,201,5,888639351
85387,943,79,5,888639019
83794,943,42,5,888639042
84853,943,182,5,888639066
91861,943,56,5,888639269
98422,943,672,5,888640125
94429,943,239,5,888639867
80701,943,928,5,875502074


In [24]:
# pandas의 as_matrix() 함수로 dataframe을 numpy array로 바꾼다
ratings_train = ratings_train.as_matrix(columns = ['user id', 'movie id', 'rating'])
ratings_test = ratings_test.as_matrix(columns = ['user id', 'movie id', 'rating'])

In [25]:
# numpy array로 바꾸어도 shape는 그대로 유지된다
print(ratings_train.shape)
print(ratings_test.shape)

(99832, 3)
(167, 3)


In [37]:
print(ratings_test)

[[ 943  721    5]
 [ 943  614    5]
 [ 943  201    5]
 [ 943   79    5]
 [ 943   42    5]
 [ 943  182    5]
 [ 943   56    5]
 [ 943  672    5]
 [ 943  239    5]
 [ 943  928    5]
 [ 943   69    5]
 [ 943  282    5]
 [ 943  194    5]
 [ 943  196    5]
 [ 943  233    5]
 [ 943   12    5]
 [ 943  471    5]
 [ 943  184    5]
 [ 943  508    5]
 [ 943   55    5]
 [ 943  127    5]
 [ 943   92    5]
 [ 943  205    5]
 [ 943  187    5]
 [ 943  186    5]
 [ 943  943    5]
 [ 943   98    5]
 [ 943   64    5]
 [ 943    2    5]
 [ 943  485    5]
 [ 943  173    5]
 [ 943  475    5]
 [ 943  100    5]
 [ 943  195    4]
 [ 943  172    4]
 [ 943  356    4]
 [ 943   28    4]
 [ 943   24    4]
 [ 943   22    4]
 [ 943  200    4]
 [ 943   68    4]
 [ 943  367    4]
 [ 943  546    4]
 [ 943   41    4]
 [ 943  763    4]
 [ 943  210    4]
 [ 943  816    4]
 [ 943  232    4]
 [ 943  732    4]
 [ 943  685    4]
 [ 943  559    4]
 [ 943  655    4]
 [ 943  161    4]
 [ 943  219    4]
 [ 943  824    4]
 [ 943   7

In [33]:
num_recommendations = 5    # 추천하고 싶은 영화의 개수 - 사용자가 임의로 지정할 수 있다

In [34]:
favored_items = []
for i in range(num_recommendations):
    favored_items.append(ratings_test[i][1])

In [35]:
favored_items    # user 943이 선호한 아이템 목록

[721, 614, 201, 79, 42]

## Item-item similarity calculation
- 각 영화 간의 유사도(거리)를 계산한다
- 유사도의 기준은 장르(action, adventure, animation, crime 등)의 유사성이다
- 결과로 1682 X 1682 크기의 행렬이 반환된다

In [39]:
def EucledianDist(item1, item2):    # 두 영화 간의 거리를 계산하기 위한 함수를 정의한다
    s = 0
    v1 = item1[5:]
    v2 = item2[5:]
    
    for i in range(len(v1)):
        temp = (v1[i]-v2[i])*(v1[i]-v2[i])    # 각 원소를 서로 뺀 후에 제곱근 해 모두 더한다
        s += temp
    return math.sqrt(s)

In [40]:
items = items.as_matrix()    # items 데이터프레임을 numPy 배열로 변환한다

In [41]:
items

array([[1, 'Toy Story (1995)', '01-Jan-1995', ..., 0, 0, 0],
       [2, 'GoldenEye (1995)', '01-Jan-1995', ..., 1, 0, 0],
       [3, 'Four Rooms (1995)', '01-Jan-1995', ..., 1, 0, 0],
       ..., 
       [1680, 'Sliding Doors (1998)', '01-Jan-1998', ..., 0, 0, 0],
       [1681, 'You So Crazy (1994)', '01-Jan-1994', ..., 0, 0, 0],
       [1682, 'Scream of Stone (Schrei aus Stein) (1991)', '08-Mar-1996',
        ..., 0, 0, 0]], dtype=object)

In [43]:
distance_matrix = np.zeros((1682, 1682))    # 1682X1682 크기의 0으로 이루어진 행렬을 생성한다

In [44]:
for i in range(len(items)):
    for j in range(len(items)):
        distance_matrix[i][j] = EucledianDist(items[i], items[j])    # 영행렬의 각 원소를 각 열번호와 행번호의 영화 간의 거리로 계산해 담는다

In [45]:
distance_matrix    # 각 영화 간의 거리를 나타내는 2차원 배열(행렬)

array([[ 0.        ,  2.44948974,  2.        , ...,  2.23606798,
         1.41421356,  2.        ],
       [ 2.44948974,  0.        ,  1.41421356, ...,  2.23606798,
         2.        ,  2.        ],
       [ 2.        ,  1.41421356,  0.        , ...,  1.73205081,
         1.41421356,  1.41421356],
       ..., 
       [ 2.23606798,  2.23606798,  1.73205081, ...,  0.        ,
         1.73205081,  1.        ],
       [ 1.41421356,  2.        ,  1.41421356, ...,  1.73205081,
         0.        ,  1.41421356],
       [ 2.        ,  2.        ,  1.41421356, ...,  1.        ,
         1.41421356,  0.        ]])

## Recommendation for user 943
- user 943을 위한 추천 영화 리스트를 작성한다
- user 943이 높게 평가한 영화들과 유사한 특성을 갖는(거리가 짧은) 영화들을 추천한다

In [52]:
recommended_movies = []
for i in favored_items:
    idx = np.argmin(distance_matrix[i])          # 각 영화와 가장 거리가 짧은 영화의 인덱스를 찾는다
    recommended_movies.append(items[idx][1])     # 인덱스를 통해 그 영화의 이름을 찾아 리스트에 첨부한다

In [53]:
# 추천 영화를 출력한다
for movie in recommended_movies:
    print(movie)

Mighty Aphrodite (1995)
Four Rooms (1995)
French Twist (Gazon maudit) (1995)
Hot Shots! Part Deux (1993)
Taxi Driver (1976)
